# Contents

0. Imports 
1. Reading data
   - 1.1 Main dataset + auxiliary dataset
   - 1.2 HDB Resale Price Index
   - 1.3 HDB Property Information
2. Data preprocessing & feature engineering
3. Modelling
   - <s>3.1 Drop nominal features</s>
   - <s>3.2 One-hot encode nominal features</s>
   - 3.3 Label-encode nominal features
4. Submission

# How to run this notebook

Run all the cells _except_ those from sections 3.1 and 3.2.

# 0. Imports

In [ ]:
!pip install -q git+https://github.com/remykarem/pandas-lightning#egg=pandas-lightning

In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import pandas_lightning
from pandas_lightning import Pipeline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

sns.set_theme()
sns.set(rc={'figure.figsize':(14.7,8.27)})

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting # for HistGradientBoostingRegressor

from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

def rmse(clf, X_test, y_true):
    return np.sqrt(mean_squared_error(y_true, clf.predict(X_test)))

# 1. Reading data

## 1.1 Main dataset + auxiliary dataset

In [ ]:
df_ = pd.read_csv("../data/train_edited.csv")

## 1.2 HDB Resale Price Index

In [ ]:
df_resale = pd.read_csv("../data/hdb-resale-price-index.csv.zip")
df_resale.rename(
    columns={"index": "price_index"},
    inplace=True
)

## 1.3 HDB Property Information

In [ ]:
df_property_ = pd.read_csv("../data/hdb-property-information.csv.zip")

df_property = df_property_.lambdas.sapply(
    
    address=(["blk_no", "street"], lambda b,s: b.str.lower() + " " + s.str.lower())
    
).lambdas.astype(
    address="index",
    residential=bool,
    commercial=bool,
    market_hawker=bool,
    miscellaneous=bool,
    multistorey_carpark=bool,
    precinct_pavilion=bool
    
).lambdas.sapply(
    rental_rooms=(["1room_rental", "2room_rental", "3room_rental", "other_room_rental"], lambda a,b,c,d: a+b+c+d)
    
).drop(
    columns=[
        "blk_no", "street", 
         # we are interested in timeless attributes
         "1room_sold", "2room_sold", "3room_sold", "4room_sold", 
         "5room_sold", "exec_sold", "multigen_sold", "studio_apartment_sold",
         # dropping because these values have been aggregated
         "1room_rental", "2room_rental", "3room_rental", "other_room_rental",
         # dropping because after merging with the main dataset, they only have 1 unique value
         "residential", "commercial", "market_hawker", "miscellaneous", "multistorey_carpark", "precinct_pavilion",
         # dropping because similar to 'town' in main dataset
         "bldg_contract_town"
        ]
    
)
df_property

# 2. Data processing & feature engineering

In [ ]:
STREET_NAME_ABBREVS = {
    "avenue": "ave",
    "bukit": "bt",
    "drive": "dr",
    "road": "rd",
    "upper": "upp",
    "south": "sth",
    "close": "cl",
    "place": "pl",
    "north": "nth",
    "street": "st",
    "central": "ctrl",
    "crescent": "cres",
    "lorong": "lor",
    "jalan": "jln",
    "gardens": "gdns",
    "tanjong": "tg",
    "kampong": "kg",
    "terrace": "terr",
    "commonwealth": "c'wealth",
}

def shorten_street_name(name: str):
    for long, short in STREET_NAME_ABBREVS.items():
        name = name.lower().replace(long,short)
    return name

def get_average_storey(storey: pd.Series):
    a, b = storey.str.split(" to ").str
    avg_storey = (b.astype(int) + a.astype(int))/2
    return avg_storey.astype("float16")

def get_flat_age(year: pd.Series, lease_commence_date: pd.Series):
    return year - lease_commence_date

def is_inauspicious(block: pd.Series):
    return block.isin(["13", "4", "44", "444"])

def is_before_covid(date: pd.Series):
    """
    Feb 2020 is the period before lockdown
    """
    return date < "20200201"

def planning_area_to_district(planning_area: pd.Series):
    """
    Information taken from the following websites:
    https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.html
    https://www.harrylau.com/basic-real-estate-knowledge-you-must-know/singapore-district-and-planning-area/
    
    Note that ['clementi', 'kallang', 'bukit merah', 'outram', 'downtown core'] span multiple districts.
    """
    
    DISTRICT_MAPPING = {
        1: ["raffles place", "cecil", "marina", "people’s park"],
        2: ["anson", "tanjong pagar"],
        3: ["queenstown", "tiong bahru", "alexandra", "outram"], # outram is 3 based on wiki
        4: ["telok blangah", "harbourfront", "sentosa", "keppel", "mount faber", "bukit merah"], # bukit merah is 2, 3, 4, 5, 9, 10
        5: ["pasir panjang", "buona vista", "dover", "west coast", "clementi new town", "clementi"], # we place clementi together with clementi new town
        6: ["high street", "beach road", "city hall"],
        7: ["middle road", "golden mile", "bugis", "rochor", "downtown core"], # downtown core is 1, 6, 7
        8: ["little india", "farrer park", "serangoon road"],
        9: ["orchard", "cairnhill", "river valley"],
        10: ["ardmore", "bukit timah", "holland road", "tanglin"],
        11: ["watten estate", "newton", "novena", "thomson"],
        12: ["balestier", "toa payoh", "serangoon"],
        13: ["macpherson", "braddell", "potong pasir"],
        14: ["geylang", "paya lebar", "eunos", "kembangan", "kallang"], # kallang is in 1, 7, 12, 13, 14, 15
        15: ["katong", "joo chiat", "amber road", "marine parade", "tanjong rhu", "meyer"],
        16: ["bedok", "upper east coast", "eastwood", "kew drive", "chai chee", "siglap"],
        17: ["loyang", "changi"],
        18: ["tampines", "pasir ris", "simei"],
        19: ["serangoon garden", "hougang", "punggol", "sengkang"],
        20: ["bishan", "ang mo kio", "braddell"],
        21: ["upper bukit timah", "ulu pandan"],
        22: ["jurong", "boon lay", "tuas", "lakeside", "jurong west", "jurong east"],
        23: ["hillview", "dairy farm", "bukit panjang", "choa chu kang", "bukit batok"],
        24: ["lim chu kang"],
        25: ["kranji", "woodgrove", "woodlands", "sungei kadut"],
        26: ["upper thomson", "springleaf"],
        27: ["yishun", "sembawang", "admiralty"],
        28: ["seletar", "yio chu kang"]
    }
    return planning_area.map_categorical_binning(DISTRICT_MAPPING).astype("category")

def is_prime_district(district: pd.Series):
    """
    https://www.propertyguru.com.sg/property-guides/ccr-ocr-rcr-region-singapore-ura-map-21045
    """
    return district.isin([9, 10, 11])

def is_core(prime_district: pd.Series, planning_area: pd.Series):
    """
    https://www.propertyguru.com.sg/property-guides/ccr-ocr-rcr-region-singapore-ura-map-21045
    """
    return prime_district | planning_area.isin(["bugis", "city hall", "sentosa", "shenton way", "tanjong pagar",
                                                "boat quay", "raffles place", "marina downtown", "suntec city"]) 

def can_use_cpf(remaining_lease: pd.Series):
    return remaining_lease < 30

def get_resale_index_quarter(year: pd.Series, date: pd.Series):
    LAG = 2
    yy = date.dt.year + ((date.dt.quarter - 1 - LAG) < 0) * -1
    qq = (date.dt.quarter - 1 - LAG) % 4 + 1
    return yy.astype(int).astype(str) + "-Q" + qq.astype(str)
    #return year.astype(int).astype(str) + "-Q" + date.dt.quarter.astype(str)
    
def get_floor_pctg(storey, max_storey):
    return storey/max_storey

def get_year_from_date(date):
    return date.dt.year.astype("float16")

def clean_flat_type(flat_type):
    return flat_type.str.replace("-", " ")

def get_flat_address(block, street_name):
    return block.str.lower() + " " + street_name.str.lower()

In [ ]:
def process(data):
    
    return data.copy(

        ).rename(
            columns={"month": "date"}
        
        ).lambdas(inplace=True).apply(
            street_name=("street_name", shorten_street_name)

        ).lambdas(inplace=True).sapply(
            flat_type=clean_flat_type,
            address=(["block", "street_name"], get_flat_address)

        ).lambdas(inplace=True).astype(
            planning_area="category",
            flat_model="category",
            subzone="category",
            region="category",
            town="category",
            date="datetime",
            flat_type=['1 room', '2 room', '3 room', '4 room', '5 room', 'executive', 'multi generation'],
        
        ).merge(
            df_property, on="address", how="left"
        
        ).lambdas(inplace=True).sapply(
            year=("date", get_year_from_date),
            avg_storey=("storey_range", get_average_storey),
            remaining_lease=(["year", "lease_commence_date"], get_flat_age),
            inauspicious=("block", is_inauspicious),
            before_covid=("date", is_before_covid),
            district=("planning_area", planning_area_to_district),
            prime_district=("district", is_prime_district),
            core_central_region=(["prime_district", "planning_area"], is_core),
            can_use_cpf=("remaining_lease", can_use_cpf),
            quarter=(["year", "date"], get_resale_index_quarter),
            floor_pctg=(["avg_storey", "max_floor_lvl"], get_floor_pctg)
        
        ).merge(
            df_resale, on="quarter", how="left"
        
        )

In [ ]:
def drop_low_feature_importance(data):
    # Based on LGR
    
    return data.drop(
        columns=[
            # 1 unique value
            "elevation", "eco_category",
            # low importance
            "can_use_cpf", "core_central_region", "core_central_region", "region", "prime_district"]
    )


def drop_similar_info(data):
    
    return data.drop(
        columns=[
                 "lease_commence_date",
                 # Dropping the following *_1km and *_2km features because *_ndist features have higher importance (10k based on LGR)
                 "markets_1km", "malls_1km", "comm_1km", "prisch_1km", "secsch_1km", "mrt_1km",
                 "markets_2km", "malls_2km", "comm_2km", "prisch_2km", "secsch_2km", "mrt_2km",
                ]
        )

def drop_no_useful_info(data):
    
    return data.drop(
        columns=[
            "eco_category", "elevation", 
                 "block", 
                 "lease_commence_date", 
                 "latitude", "longitude",
                ]
        )


In [ ]:
# pipeline = Pipeline()
# pipeline.add(process, drop_low_feature_importance)

pipeline2 = Pipeline()
pipeline2.add(process, drop_low_feature_importance, drop_similar_info)

In [ ]:
df = pipeline(df_)
df

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
df.tests.info(pctg=False).sort_values(by="dtype")

In [ ]:
df.quickplot(
    numerical=["resale_price", "date"],
#     categorical=["flat_model"]

).lineplot()

# 3. Modelling

## 3.1 Models for dropping nominal features

* Linear Regression
* Decision Tree
* Random Forest
* XGBRegressor
* GradientBoostingRegressor

RMSE: 50k-55k

In [ ]:
X, y, metadata = df.dataset(pipelines=[submission_pipeline]).to_X_y(
    target="resale_price",
    nominal="drop",
    nans="remove",
    nominal_max_cardinality=100
)

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, coef in zip(X.columns, model.coef_):
    print(f"{feat:20}: {coef}")

Decision Tree

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

Random Forest

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

XGBRegressor

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

## 3.2 Models for one-hot encoding nominal features

* Linear Regression
* Decision Tree
* Random Forest
* XGBRegressor
* GradientBoostingRegressor

RMSE: 26k-28k

In [ ]:
X, y, metadata = df.dataset(pipelines=[onehot_pipeline]).to_X_y(
    target="resale_price",
    nominal="one-hot",
    nans="remove",
    nominal_max_cardinality=100
)

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, coef in zip(X.columns, model.coef_):
    print(f"{feat:20}: {coef}")

Decision Tree

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators=10_000, min_samples_leaf=10)
rf.fit(X_train, y_train)
score = rmse(rf, X_test, y_test)
print(score)

# for feat, imp in zip(X.columns, model.feature_importances_):
#     print(f"{feat:20}: {imp}")

XGBRegressor

In [ ]:
model = XGBRegressor(n_estimators=10_000)
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

XGBRFRegressor

In [ ]:
model = XGBRFRegressor()
model.fit(X_train, y_train)
score = rmse(model, X_test, y_test)
print(score)

for feat, imp in zip(X.columns, model.feature_importances_):
    print(f"{feat:20}: {imp}")

## 3.3 Models for label-encoding nominal features

* HistGradientBoostingRegressor
* CatBoost
* LGBMRegressor

RMSE: 19k-21k

In [ ]:
X, y, metadata = pipeline2(df_).dataset(pipelines=[pipeline2]).to_X_y(
    target="resale_price",
    nominal="label",
    nans="keep",
    nominal_max_cardinality=200
)

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5228)

"Overfit"

In [ ]:
hgb = HistGradientBoostingRegressor(
    max_iter=200_000,
    categorical_features=metadata["nominal"]["col_indices"],
    l2_regularization=0.05
)

In [ ]:
hgb.fit(X_train, y_train)

In [ ]:
rmse(hgb, X_test, y_test)

"Generalise"

In [ ]:
hgb2 = HistGradientBoostingRegressor(
    max_iter=100_000,
    categorical_features=metadata["nominal"]["col_indices"],
    max_bins=160,
    min_samples_leaf=30,
    l2_regularization=0.05
)
hgb2.fit(X_train, y_train)

In [ ]:
rmse(hgb2, X_test, y_test)

"Super generalise"

In [ ]:
hgb3 = HistGradientBoostingRegressor(
    max_iter=100_000,
    categorical_features=metadata["nominal"]["col_indices"],
    max_bins=160,
    min_samples_leaf=50,
    l2_regularization=0.05
)
hgb3.fit(X_train, y_train)

In [ ]:
rmse(hgb3, X_test, y_test)

"Generalise with CV"

In [ ]:
param_grid = {
    "max_iter": [200,500],  # no: 100
    "categorical_features": [metadata["nominal"]["col_indices"]],
    "min_samples_leaf": [20,30,40,50]
}
clf = GridSearchCV(estimator=HistGradientBoostingRegressor(),
                   param_grid=param_grid,
                   cv=5,
                   scoring="neg_mean_squared_error")
clf.fit(X, y)

In [ ]:
rmse(clf, X_test, y_test)

In [ ]:
param_grid = {
    "max_iter": [200,500,1000],  # no: 100
    "categorical_features": [metadata["nominal"]["col_indices"]],
    "min_samples_leaf": [20,30,40,50]
}
clf2 = GridSearchCV(estimator=HistGradientBoostingRegressor(),
                   param_grid=param_grid,
                   cv=5,
                   scoring="neg_mean_squared_error")
clf2.fit(X, y)
rmse(clf2, X_test, y_test)

In [ ]:
param_grid = {
    "max_iter": [500,1000,1500],  
    "categorical_features": [metadata["nominal"]["col_indices"]],
    "min_samples_leaf": [40,50,60]
}
clf3 = GridSearchCV(estimator=HistGradientBoostingRegressor(),
                   param_grid=param_grid,
                   cv=5,
                   scoring="neg_mean_squared_error")
clf3.fit(X, y)
rmse(clf3, X_test, y_test)

In [ ]:
clf2.best_estimator_

In [ ]:
lgr = LGBMRegressor(n_estimators=10_000)
lgr.fit(X_train, y_train, categorical_feature=metadata["nominal"]["col_indices"])
score = rmse(lgr, X_test, y_test)
print(score)

for feat, imp in zip(X_train.columns, lgr.feature_importances_):
    print(f"{feat:20}: {imp}")

In [ ]:
lgr2 = LGBMRegressor(n_estimators=20_000)
lgr2.fit(X_train, y_train, categorical_feature=metadata["nominal"]["col_indices"])
score = rmse(lgr2, X_test, y_test)
print(score)

for feat, imp in zip(X_train.columns, lgr.feature_importances_):
    print(f"{feat:20}: {imp}")

In [ ]:
lgr.feature_importances_

In [ ]:
class Ensemble:
    
    def __init__(self, weights: list, *models):
        self.models = models
        self.weights = weights
        if len(self.models) != len(self.weights):
            raise ValueError("No. of weights and models must be the same")
        if sum(self.weights) != 1:
            raise ValueError("Weights must sum to 1")
            
    def predict(self, X):
        y = [weight*model.predict(X) 
             for weight, model in zip(self.weights, self.models)]
        y = np.vstack(y)
        mean = y.sum(axis=0)
        return mean

In [ ]:
ens = Ensemble([0.2, 0.2, 0.2, 0.2, 0.2], clf2, hgb, hgb2, clf, lgr)

In [ ]:
rmse(ens, X_test, y_test)

In [ ]:
pipeline2.add(ens.predict)

# 4. Submission

In [ ]:
test__ = pd.read_csv("../data/test.csv")

test_ = pd.read_csv("../data/test_edited_v2.csv")

for col in ["lease_commence_date", "storey_range", "block", "eco_category", 'longitude', 'latitude', 'elevation']:
    test_[col] = test__[col]

In [ ]:
preds = pipeline2(test_)
preds

In [ ]:
submission = pd.DataFrame({"Predicted": preds})
submission.to_csv("submission.csv", index_label="Id")